In [ ]:
#Bibliotecas utilizadas
import cv2
import math
import numpy as np
import matplotlib.pyplot as plt

1) Implement a function that performs global and local histogram equalization of an image and test it on images flower_bee and fig3.15(a). Discuss the results by evaluating the histograms. Calculate the sample mean and sample variance of the images, before and after equalization. Show, for each case, the histogram before and after applying the methods. Is it possible to relate the results to the images? Explain.

In [ ]:
def calcula_histograma (img):
  histogram = [0] *256
  for i in range (img.shape[0]):
    for j in range (img.shape[1]):
      intensidade = int(img[i,j])
      histogram [intensidade] = histogram[intensidade] + 1
  return histogram

def equalizacao_global (img):
  histogram = calcula_histograma(img)
  img_equalized = [0]*256
  img_equalized[0] = histogram[0]
  for i in range(1,256):


image = cv2.imread('/content/drive/MyDrive/Material_mestrado/Processamento digital de imagens/Lista 1/flower_bee.png', cv2.IMREAD_GRAYSCALE)
generate_histogram_function(image)


NameError: name 'generate_histogram_function' is not defined

2) Reproduce Example 3.12 of the textbook (3rd edition).

3) Make a routine that implements an NxN (N must be an odd number) spatial
convolution mask. You must discuss possible solutions for padding the borders and
implement one of them. Evaluate the implemented routine on the lena.tif image, for
the following cases: a) Low-Pass Filter, b) Laplacian filter (adding the original
image to the result), c) Sobel filter (pass the two masks and return the image of the
gradient magnitude). Show the obtained images and analyze them.

4) Write a routine that implements in the frequency domain: a) a Butterworth low-pass filter of order n=1 and apply this filter on the image Fig3.35(a).jpg for twovalues of cutoff frequency (D0); b) a Butterworth low-pass filter of order n=8 on the same image and same cutoff frequencies; c) an ideal filter on the same image and same cutoff frequencies. Compare and comment on the results.

5) For the image mar-il.gif, correct the problem of poor illumination in 2 different ways (assume the multiplicative model between reflectance and illumination and discuss the solutions).

6) Reduce the image frexp_1.png by alternately eliminating rows and columns.
Compare the original image with the reduced one. What happened to it? What
procedure could you apply to reduce this effect? Apply the solution devised to
avoid this effect in the reduced image. Compare the new image with the other two
and evaluate the result.

7) Reproduce Example 4.24 of the textbook (3rd edition).

8) Apply to image Fig5.26(a).jpg the blur of Example 5.10 the textbook (3rd edition) and add a Gaussian noise of zero mean and variance of 200. Use the Wiener filtering and inverse filtering to try to recover the image. Calculate the PSNR of the recovered images, present them, and comment on methodologies and results. PSNR (Peak Signal to Noise Ratio) is computed using: where I(i, j) and K(i, j) are the original and corrupted images,respectively, and MAX is the highest gray level value of the original image. Use log10 to calculate PSNR.

9) For the images noise1.jpg and noise2.jpg, apply the filters mean 5x5, median 5x5 and the adaptive median filter (ranging from dimensions 3x3 to dimensions 11x11) to treat the noise. Compare with the original image.jpg by calculating the PSNR. Use log10 to calculate the PSNR. Compare the results and discuss.



##Theoretical questions:


1) Explain with your words the process to obtain a digital image from a continuous
image (consider that (x,y) coordinates and f amplitude are continuous). Make the
considerations regarding spatial and intensity resolution.


2) How would you explain spatial frequency in an image?


3) List and explain the problems related to filtering in the frequency domain and the use of DFT for this purpose. What can be done to reduce/avoid these problems?


4) Explain the optimum notch filtering procedure and in which situations it is interesting to use it.